# Open AI

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/openai.ipynb)

Ce bloc-note Jupyter explore les différentes capacités d'[OpenAI](https://openai.com/) à travers son API en langage Python. D'une manière générale, on préférera utiliser les modèles d'Open AI à travers les API de [LangChain](./langchain.ipynb) ou [LlamaIndex](./llamaindex.ipynb) pour les applications de génération augmentée de récupération (RAG).

Docs:
- https://platform.openai.com/overview
- https://github.com/openai/openai-python

## Installation des SDK

In [ ]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -qU -r ../requirements.txt

# À installer dans tous les cas pour Google Colab et Github
if 'google.colab' in sys.modules or 'CI' in os.environ:
    !pip3 install -qU openai
    !pip3 install -qU openai-agents

In [1]:
work_directory = './openai'

!mkdir -p $work_directory

## Chargement de la Clé pour OpenAI

Il vous faut obtenir d'Open AI une clé pour exécuter ce bloc-note Jupyter. Vous pouvez consulter [Where do I find my API key?](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key). Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [6]:
if 'google.colab' in sys.modules:
  from google.colab import userdata
  openai_api_key = userdata.get('OPENAI_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # lire le fichier .env local
  openai_api_key  = os.environ['OPENAI_API_KEY']

In [7]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

## Vectorisation (embeddings)

In [ ]:
def get_embedding(text):
    return client.embeddings.create(input=[text], model='text-embedding-3-small').data[0].embedding

emb_1 = get_embedding('Le petit chat court après la balle.')
print(emb_1)

[0.0028962292708456516, 0.03444449231028557, -0.0035152819473296404, -0.0806436687707901, -0.05638500303030014, -0.0032899058423936367, -0.009044313803315163, 0.007375945337116718, -0.0038431016728281975, -0.003646995173767209, 0.021015590056777, -0.0365753211081028, -0.0330863818526268, 0.024656731635332108, 0.004536791704595089, 0.02510163001716137, 0.00755156297236681, -0.02164781466126442, -0.029199376702308655, 0.01495092362165451, -0.018873054534196854, -0.005154380574822426, 2.1380545149440877e-05, 0.0008971139905042946, 0.030697980895638466, 0.003690899582579732, 0.00421775272116065, -0.057508956640958786, 0.053668782114982605, 0.04720604792237282, 0.03289905562996864, -0.03278197720646858, -0.04884514957666397, -0.04533279314637184, -0.08003485947847366, 0.024656731635332108, 0.004691920708864927, -0.016250494867563248, 0.043974682688713074, -0.002542066853493452, -0.004150432534515858, 0.004501668270677328, 0.010185829363763332, -0.020242871716618538, 0.015711933374404907, -0

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

emb_2 = get_embedding('Le petit chien court après la balle.')
emb_3 = get_embedding('La voiture est arrêtée au feu rouge.')

print(cosine_similarity(emb_1, emb_2))
print(cosine_similarity(emb_2, emb_3))


0.8641581018200647
0.30310007666028865


## Conversations Chat

In [ ]:
completion = client.chat.completions.create(
  model='gpt-3.5-turbo',
  messages=[
    {'role': 'system', 'content': 'Tu es un professeur d\'histoire.'},
    {'role': 'user', 'content': 'Quand la première guerre mondiale a-t-elle commencé?'},
    {'role': 'assistant', 'content': 'La première guerre mondiale a commencé en 1914.'},
    {'role': 'user', 'content': 'Quand a-t-elle fini?'}
  ]
)

response = completion.choices[0].message.content
print(response)

La première guerre mondiale s'est terminée en 1918.


## Synthèse vocale (text-to-speech)

In [ ]:
from IPython.display import Audio 

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

speech_file_path = f'{work_directory}/speech.mp3'

response = client.audio.speech.create(
  model='tts-1',
  voice='shimmer',
  input=response
)
response.stream_to_file(speech_file_path)
Audio(speech_file_path)

## Reconnaissance vocale (speech-to-text)

In [ ]:
audio_file= open(speech_file_path, 'rb')
transcription = client.audio.transcriptions.create(
  model='whisper-1', 
  file=audio_file
)
print(transcription.text)

La Première Guerre mondiale s'est terminée en 1918.


## Génération d'images

In [8]:
from IPython.display import Image

response = client.images.generate(
  model='dall-e-3',
  prompt='Un robot de la première guerre mondiale',
  size='1024x1024',
  quality='standard',
  n=1,
)
image_url = response.data[0].url
Image(url=image_url)

## Agents

In [ ]:
import requests
from agents import Agent, Runner, function_tool
from pydantic import BaseModel

class Weather(BaseModel):
    location: str
    temperature: float
    temperature_range: str
    humidity: int
    wind_speed: float
    wind_direction: int
    weather_description: str
    weather_code: int
    feels_like: float
    pressure: float
    visibility: float
    uv_index: float
    timestamp: str

@function_tool
def get_weather(lat: float, lon: float) -> Weather:
    print("[debug] get_weather called")
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,apparent_temperature,pressure_msl,surface_pressure,wind_speed_10m,wind_direction_10m,weather_code,visibility,uv_index&daily=temperature_2m_max,temperature_2m_min&timezone=auto")
    data = response.json()
    current = data['current']
    daily = data['daily']
    # Weather code descriptions (WMO codes)
    weather_descriptions = {
        0: "Clear sky",
        1: "Mainly clear",
        2: "Partly cloudy",
        3: "Overcast",
        45: "Fog",
        48: "Depositing rime fog",
        51: "Light drizzle",
        53: "Moderate drizzle",
        55: "Dense drizzle",
        61: "Slight rain",
        63: "Moderate rain",
        65: "Heavy rain",
        71: "Slight snow",
        73: "Moderate snow",
        75: "Heavy snow",
        95: "Thunderstorm"
    }
    return Weather(
        location=f"Location ({lat}, {lon})",
        temperature=current['temperature_2m'],
        temperature_range=f"{daily['temperature_2m_min'][0]}-{daily['temperature_2m_max'][0]}°C",
        humidity=current['relative_humidity_2m'],
        wind_speed=current['wind_speed_10m'],
        wind_direction=current['wind_direction_10m'],
        weather_description=weather_descriptions.get(current['weather_code'], "Unknown"),
        weather_code=current['weather_code'],
        feels_like=current['apparent_temperature'],
        pressure=current['pressure_msl'],
        visibility=current['visibility'],
        uv_index=current['uv_index'],
        timestamp=current['time']
    )

agent = Agent(
    name="WeatherAgent",
    instructions="Tu es un assistant météo qui répond en Français. Tu refuses polimment de répondre aux questions qui ne sont pas liées à la météo.",
    tools=[get_weather],
)

result = await Runner.run(agent, "Quel est le temps à Paris?")
print(result.final_output)

[debug] get_weather called
Il fait actuellement 19,7°C à Paris avec un ciel couvert. L'humidité est de 74% et il y a un vent de 10,7 km/h venant du nord (direction 346°). La température ressentie est de 19,8°C. La pression atmosphérique est de 1015,3 hPa et la visibilité est de 33,5 km. Profitez de votre journée !


In [ ]:
# Ménage
!rm -rf $work_directory